In [15]:
import numpy as np
import pandas as pd
import scipy.io
from pypower.api import ext2int, int2ext, makeYbus
import numpy as np
import matplotlib.pyplot as plt
PATH = "/storage/home/hcoda1/6/btaheri6/p-dmolzahn6-0/DistFlow"

TEST_CASE           =   '_small'


def load_mat():
    mat = scipy.io.loadmat(f"{PATH}/data/case{TEST_CASE}.mat")

    bus = mat['ans']['bus'][0][0]
    baseMVA = mat['ans']['baseMVA'][0][0][0][0]
    gen = mat['ans']['gen'][0][0]
    branch = mat['ans']['branch'][0][0]
    keys = ['baseMVA', 'bus', 'gen', 'branch']
    values = [baseMVA, bus, gen, branch]
    ppc = dict(zip(keys, values))
    ppc = ext2int(ppc)
    baseMVA, bus, gen, branch = ppc['baseMVA'], ppc['bus'], ppc['gen'], ppc['branch']
    Ybus, Yf, Yt = makeYbus(baseMVA, bus, branch)

    return baseMVA, bus, gen, branch, Ybus, Yf, Yt, ppc

def calculate_just_loads (baseMVA, bus):
    Just_load = bus[np.logical_or(bus[:, 2] != 0, bus[:, 3] != 0)]
    load_buses=Just_load[:,0]
    load_buses=np.reshape(load_buses,(len(load_buses),1))

    P_Load  = Just_load [:,2]/baseMVA
    Q_Load  = Just_load [:,3]/baseMVA
    P_load  = np.reshape(P_Load,(len(load_buses),1))    
    Q_load  = np.reshape(Q_Load,(len(load_buses),1)) 

    return P_load, Q_load

In [9]:
baseMVA, bus, gen, branch, Ybus, Yf, Yt, ppc = load_mat()
P_load, Q_load = calculate_just_loads (baseMVA, bus)

TEST_CASE           =   '33bw_mod'

N=1000
def generate_scenarios(P_load, Q_load, N):
    # Initialize the scenarios matrix
    P_load_scenarios = np.zeros((N, P_load.shape[0]))
    Q_load_scenarios = np.zeros((N, Q_load.shape[0]))
    
    # For every non-zero element in P_load and Q_load, generate N scenarios using a normal distribution.
    for idx in range(P_load.shape[0]):
        if P_load[idx, 0] != 0:
            P_load_scenarios[:, idx] = np.random.normal(P_load[idx, 0], abs(0.15 * P_load[idx, 0]), N)
        if Q_load[idx, 0] != 0:
            Q_load_scenarios[:, idx] = np.random.normal(Q_load[idx, 0], abs(0.15 * Q_load[idx, 0]), N)
    
    return P_load_scenarios, Q_load_scenarios

P_load_scenarios, Q_load_scenarios = generate_scenarios(P_load, Q_load, N)
np.save(f"{PATH}/data/P_load_all_scenarios_{TEST_CASE}bus.npy", P_load_scenarios)
np.save(f"{PATH}/data/Q_load_all_scenarios_{TEST_CASE}bus.npy", Q_load_scenarios)

# Const Power factor

In [16]:
import numpy as np
baseMVA, bus, gen, branch, Ybus, Yf, Yt, ppc = load_mat()
P_load, Q_load = calculate_just_loads (baseMVA, bus)

TEST_CASE           =   '_small'

N=1000
def generate_scenarios(P_load, Q_load, N):
    # Calculate the initial power factor
    power_factor = P_load / np.sqrt(P_load**2 + Q_load**2)

    # Initialize the scenarios matrix
    P_load_scenarios = np.zeros((N, P_load.shape[0]))
    Q_load_scenarios = np.zeros((N, Q_load.shape[0]))

    # For every non-zero element in P_load, generate N scenarios.
    # Then calculate corresponding Q_load scenarios to maintain power factor
    for idx in range(P_load.shape[0]):
        if P_load[idx, 0] != 0:
            P_load_scenarios[:, idx] = np.random.normal(P_load[idx, 0], abs(0.15 * P_load[idx, 0]), N)
            # Calculate Q for each P scenario while keeping power factor constant
            Q_load_scenarios[:, idx] = P_load_scenarios[:, idx] * np.tan(np.arccos(power_factor[idx, 0]))

    return P_load_scenarios, Q_load_scenarios
P_load_scenarios, Q_load_scenarios = generate_scenarios(P_load, Q_load, N)
np.save(f"{PATH}/data/P_load_all_scenarios_{TEST_CASE}bus.npy", P_load_scenarios)
np.save(f"{PATH}/data/Q_load_all_scenarios_{TEST_CASE}bus.npy", Q_load_scenarios)

In [9]:
import numpy as np


N=1000
def generate_scenarios(P_load, Q_load, N):
    # Calculate the initial power factor
    power_factor = P_load / np.sqrt(P_load**2 + Q_load**2)

    # Initialize the scenarios matrix
    P_load_scenarios = np.zeros((N, P_load.shape[0]))
    Q_load_scenarios = np.zeros((N, Q_load.shape[0]))

    # For every non-zero element in P_load, generate N scenarios.
    # Then calculate corresponding Q_load scenarios to maintain power factor
    for idx in range(P_load.shape[0]):
        if P_load[idx, 0] != 0:
            P_load_scenarios[:, idx] = np.random.normal(P_load[idx, 0], abs(0.3 * P_load[idx, 0]), N)
            # Calculate Q for each P scenario while keeping power factor constant
            Q_load_scenarios[:, idx] = P_load_scenarios[:, idx] * np.tan(np.arccos(power_factor[idx, 0]))

    return P_load_scenarios, Q_load_scenarios

def generate_scenarios_neg(P_load, Q_load, N):
    # Calculate the initial power factor
    power_factor = P_load / np.sqrt(P_load**2 + Q_load**2)

    # Initialize the scenarios matrix
    P_load_scenarios = np.zeros((N, P_load.shape[0]))
    Q_load_scenarios = np.zeros((N, Q_load.shape[0]))

    # For every non-zero element in P_load, generate N scenarios.
    # Then calculate corresponding Q_load scenarios to maintain power factor
    for idx in range(P_load.shape[0]):
        if P_load[idx, 0] != 0:
            P_load_scenarios[:, idx] = np.random.normal(-P_load[idx, 0], abs(0.3 * P_load[idx, 0]), N)
            # Calculate Q for each P scenario while keeping power factor constant
            Q_load_scenarios[:, idx] = P_load_scenarios[:, idx] * np.tan(np.arccos(power_factor[idx, 0]))

    return P_load_scenarios, Q_load_scenarios


P_load_scenarios_pos, Q_load_scenarios_pos = generate_scenarios(P_load, Q_load, N)
P_load_scenarios_neg, Q_load_scenarios_neg = generate_scenarios_neg(P_load, Q_load, N)

P_load_scenarios = np.concatenate ((P_load_scenarios_pos,P_load_scenarios_neg))
Q_load_scenarios = np.concatenate ((Q_load_scenarios_pos,Q_load_scenarios_neg))


np.save(f"{PATH}/data/P_load_all_scenarios_{TEST_CASE}bus.npy", P_load_scenarios)
np.save(f"{PATH}/data/Q_load_all_scenarios_{TEST_CASE}bus.npy", Q_load_scenarios)

In [17]:
import numpy as np
import pandas as pd
import array
import csv
import math
import os.path
import random
import time
from joblib import Parallel, delayed
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
import scipy.io
from pypower.api import ext2int, int2ext, makeYbus
PATH = "/storage/home/hcoda1/6/btaheri6/p-dmolzahn6-0/DistFlow"


TEST_CASE           =   '_small'


def load_mat():
    mat = scipy.io.loadmat(f"{PATH}/data/case{TEST_CASE}.mat")
    bus = mat['ans']['bus'][0][0]
    baseMVA = mat['ans']['baseMVA'][0][0][0][0]
    gen = mat['ans']['gen'][0][0]
    branch = mat['ans']['branch'][0][0]
    keys = ['baseMVA', 'bus', 'gen', 'branch']
    values = [baseMVA, bus, gen, branch]
    ppc = dict(zip(keys, values))
    ppc = ext2int(ppc)
    baseMVA, bus, gen, branch = ppc['baseMVA'], ppc['bus'], ppc['gen'], ppc['branch']
    Ybus, Yf, Yt = makeYbus(baseMVA, bus, branch)

    return baseMVA, bus, gen, branch, Ybus, Yf, Yt, ppc


def load_gen_data(filename):
    gen_data = pd.read_csv(filename, sep=',')
    return gen_data


def process_bus_and_gen(bus, gen, baseMVA, branch, ppc):
    Just_load = bus[np.logical_or(bus[:, 2] != 0, bus[:, 3] != 0)]
    load_buses=Just_load[:,0]
    load_buses=np.reshape(load_buses,(len(load_buses),1))

    n_buses = len(ppc['bus'])
    n_gens = len(ppc['gen'])
    n_branches = len(ppc['branch'])
    r = branch[:, 2]
    x = branch[:, 3]
    gen_buses = gen[:, 0]
    gen_values = gen[:, 1]
    Q_values = gen[:, 2]


    Pgen = np.zeros(n_buses)
    Qgen = np.zeros(n_buses)

    for i in range(len(gen_buses)):
        bus_idx = int(gen_buses[i])  
        Pgen[bus_idx] += gen_values[i]
        Qgen[bus_idx] += Q_values[i]

    #Depends on ...    
    Pgen  = Pgen / baseMVA
    Qgen  = Qgen / baseMVA

    Pgen = np.reshape(Pgen,(n_buses,1))    
    Qgen = np.reshape(Qgen,(n_buses,1))    

    ref_bus = np.where(bus[:, 1] == 3)[0][0]

    return load_buses, n_buses, n_gens, n_branches, r, x, Pgen,Qgen, ref_bus


baseMVA, bus, gen, branch, Ybus, Yf, Yt, ppc = load_mat()
load_buses, n_buses, n_gens, n_branches, r, x, Pgen,Qgen, ref_bus = process_bus_and_gen(bus, gen, baseMVA, branch, ppc)

def compute_and_save_pinj():
    
    baseMVA, bus, gen, branch, _, _, _, ppc = load_mat()
    P_load_all_scenarios = np.load(f"{PATH}/data/P_load_all_scenarios_{TEST_CASE}bus.npy", allow_pickle=True)
    Q_load_all_scenarios = np.load(f"{PATH}/data/Q_load_all_scenarios_{TEST_CASE}bus.npy", allow_pickle=True)

    load_buses, n_buses, _, _, _, _, Pgen,Qgen, ref_bus = process_bus_and_gen(bus, gen, baseMVA, branch, ppc)

    # Pre-calculate P_inj for all scenarios
    P_inj_all_scenarios = []
    for s in range(P_load_all_scenarios.shape[0]): 
    #for s in range(P_load_all_scenarios.shape[1]-1): 
        aaa = P_load_all_scenarios[s, :]
        #aaa = P_load_all_scenarios.iloc[:, s+1]

        Demand = np.zeros(n_buses)
        for i in range(len(load_buses)):
            #bus_idx = int(load_buses[i])
            bus_idx = int(load_buses[i][0])  # assuming load_buses[i] is a 1D array with one element

            Demand[bus_idx] = aaa[i]
        Demand = np.reshape(Demand, (n_buses, 1))
        P_inj = Pgen - Demand
        P_inj = np.delete(P_inj, ref_bus, 0)
        P_inj = np.asmatrix(P_inj)
        P_inj_all_scenarios.append(P_inj)
        
    Q_inj_all_scenarios = []
    for s in range(Q_load_all_scenarios.shape[0]): 
    #for s in range(Q_load_all_scenarios.shape[1]-1): 
        aaa = Q_load_all_scenarios[s, :]
        #aaa = Q_load_all_scenarios.iloc[:, s+1]

        Demand = np.zeros(n_buses)
        for i in range(len(load_buses)):
            #bus_idx = int(load_buses[i])
            bus_idx = int(load_buses[i][0])  # assuming load_buses[i] is a 1D array with one element

            Demand[bus_idx] = aaa[i]
        Demand = np.reshape(Demand, (n_buses, 1))
        Q_inj = Qgen - Demand
        Q_inj = np.delete(Q_inj, ref_bus, 0)
        Q_inj = np.asmatrix(Q_inj)
        Q_inj_all_scenarios.append(Q_inj)        
        
    # Save to npy file
    np.save(f"{PATH}/data/P_inj_all_scenarios_{TEST_CASE}bus.npy", P_inj_all_scenarios)
    np.save(f"{PATH}/data/Q_inj_all_scenarios_{TEST_CASE}bus.npy", Q_inj_all_scenarios)

# Call the function
compute_and_save_pinj()


# High Load

In [10]:
import numpy as np

N = 30  # Number of scenarios for high loads
k_values_positive = np.arange(1, 2.00001, 1/14)
k_values_negative = -np.flip(k_values_positive)  # Generate negative values by flipping the positive ones

k_values = np.concatenate(( k_values_negative, k_values_positive))

def generate_high_load_scenarios(P_load, Q_load, N, k_values):
    # Initialize the scenarios matrix
    P_load_scenarios = np.zeros((N, P_load.shape[0]))
    Q_load_scenarios = np.zeros((N, Q_load.shape[0]))

    # For every non-zero element in P_load, generate N scenarios for high loads with specified k values.
    for idx in range(P_load.shape[0]):
        if P_load[idx, 0] != 0:
            for i, k in enumerate(k_values[:N]):
                P_load_scenarios[i, idx] = k * P_load[idx, 0]
                Q_load_scenarios[i, idx] = k * Q_load[idx, 0]

    return P_load_scenarios, Q_load_scenarios


P_load_high_scenarios, Q_load_high_scenarios = generate_high_load_scenarios(P_load, Q_load, N, k_values)

np.save(f"{PATH}/data/High_P_load_scenarios_{TEST_CASE}bus.npy", P_load_high_scenarios)
np.save(f"{PATH}/data/High_Q_load_scenarios_{TEST_CASE}bus.npy", Q_load_high_scenarios)

In [7]:
import numpy as np
import pandas as pd
import array
import csv
import math
import os.path
import random
import time
from joblib import Parallel, delayed
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
import scipy.io
from pypower.api import ext2int, int2ext, makeYbus
PATH = "/storage/home/hcoda1/6/btaheri6/p-dmolzahn6-0/DistFlow"


TEST_CASE           =   '123'


def load_mat():
    mat = scipy.io.loadmat(f"{PATH}/data/case{TEST_CASE}.mat")
    bus = mat['ans']['bus'][0][0]
    baseMVA = mat['ans']['baseMVA'][0][0][0][0]
    gen = mat['ans']['gen'][0][0]
    branch = mat['ans']['branch'][0][0]
    keys = ['baseMVA', 'bus', 'gen', 'branch']
    values = [baseMVA, bus, gen, branch]
    ppc = dict(zip(keys, values))
    ppc = ext2int(ppc)
    baseMVA, bus, gen, branch = ppc['baseMVA'], ppc['bus'], ppc['gen'], ppc['branch']
    Ybus, Yf, Yt = makeYbus(baseMVA, bus, branch)

    return baseMVA, bus, gen, branch, Ybus, Yf, Yt, ppc


def load_gen_data(filename):
    gen_data = pd.read_csv(filename, sep=',')
    return gen_data


def process_bus_and_gen(bus, gen, baseMVA, branch, ppc):
    Just_load = bus[np.logical_or(bus[:, 2] != 0, bus[:, 3] != 0)]
    load_buses=Just_load[:,0]
    load_buses=np.reshape(load_buses,(len(load_buses),1))

    n_buses = len(ppc['bus'])
    n_gens = len(ppc['gen'])
    n_branches = len(ppc['branch'])
    r = branch[:, 2]
    x = branch[:, 3]
    gen_buses = gen[:, 0]
    gen_values = gen[:, 1]
    Q_values = gen[:, 2]


    Pgen = np.zeros(n_buses)
    Qgen = np.zeros(n_buses)

    for i in range(len(gen_buses)):
        bus_idx = int(gen_buses[i])  
        Pgen[bus_idx] += gen_values[i]
        Qgen[bus_idx] += Q_values[i]

    #Depends on ...    
    Pgen  = Pgen / baseMVA
    Qgen  = Qgen / baseMVA

    Pgen = np.reshape(Pgen,(n_buses,1))    
    Qgen = np.reshape(Qgen,(n_buses,1))    

    ref_bus = np.where(bus[:, 1] == 3)[0][0]

    return load_buses, n_buses, n_gens, n_branches, r, x, Pgen,Qgen, ref_bus


baseMVA, bus, gen, branch, Ybus, Yf, Yt, ppc = load_mat()
load_buses, n_buses, n_gens, n_branches, r, x, Pgen,Qgen, ref_bus = process_bus_and_gen(bus, gen, baseMVA, branch, ppc)

def compute_and_save_pinj():
    
    baseMVA, bus, gen, branch, _, _, _, ppc = load_mat()
    P_load_all_scenarios = np.load(f"{PATH}/data/High_P_load_scenarios_{TEST_CASE}bus.npy", allow_pickle=True)
    Q_load_all_scenarios = np.load(f"{PATH}/data/High_Q_load_scenarios_{TEST_CASE}bus.npy", allow_pickle=True)

    load_buses, n_buses, _, _, _, _, Pgen,Qgen, ref_bus = process_bus_and_gen(bus, gen, baseMVA, branch, ppc)

    # Pre-calculate P_inj for all scenarios
    P_inj_all_scenarios = []
    for s in range(P_load_all_scenarios.shape[0]): 
    #for s in range(P_load_all_scenarios.shape[1]-1): 
        aaa = P_load_all_scenarios[s, :]
        #aaa = P_load_all_scenarios.iloc[:, s+1]

        Demand = np.zeros(n_buses)
        for i in range(len(load_buses)):
            #bus_idx = int(load_buses[i])
            bus_idx = int(load_buses[i][0])  # assuming load_buses[i] is a 1D array with one element

            Demand[bus_idx] = aaa[i]
        Demand = np.reshape(Demand, (n_buses, 1))
        P_inj = Pgen - Demand
        P_inj = np.delete(P_inj, ref_bus, 0)
        P_inj = np.asmatrix(P_inj)
        P_inj_all_scenarios.append(P_inj)
        
    Q_inj_all_scenarios = []
    for s in range(Q_load_all_scenarios.shape[0]): 
    #for s in range(Q_load_all_scenarios.shape[1]-1): 
        aaa = Q_load_all_scenarios[s, :]
        #aaa = Q_load_all_scenarios.iloc[:, s+1]

        Demand = np.zeros(n_buses)
        for i in range(len(load_buses)):
            #bus_idx = int(load_buses[i])
            bus_idx = int(load_buses[i][0])  # assuming load_buses[i] is a 1D array with one element

            Demand[bus_idx] = aaa[i]
        Demand = np.reshape(Demand, (n_buses, 1))
        Q_inj = Qgen - Demand
        Q_inj = np.delete(Q_inj, ref_bus, 0)
        Q_inj = np.asmatrix(Q_inj)
        Q_inj_all_scenarios.append(Q_inj)        
        
    # Save to npy file
    np.save(f"{PATH}/data/High_P_inj_all_scenarios_{TEST_CASE}bus.npy", P_inj_all_scenarios)
    np.save(f"{PATH}/data/High_Q_inj_all_scenarios_{TEST_CASE}bus.npy", Q_inj_all_scenarios)

# Call the function
compute_and_save_pinj()

# uniform scenarios

In [17]:
import numpy as np

N = 10000

def generate_scenarios_uniform(P_load, Q_load, N):
    # Calculate the initial power factor
    power_factor = P_load / np.sqrt(P_load**2 + Q_load**2)

    # Initialize the scenarios matrix
    P_load_scenarios = np.zeros((N, P_load.shape[0]))
    Q_load_scenarios = np.zeros((N, Q_load.shape[0]))

    # For every non-zero element in P_load, generate N scenarios.
    # Draw random load values from a uniform distribution in the interval (1.5 * P_load, 0)
    for idx in range(P_load.shape[0]):
        if P_load[idx, 0] != 0:
            P_load_scenarios[:, idx] = np.random.uniform(0, 1.5 * P_load[idx, 0], N)
            # Calculate Q for each P scenario while keeping power factor constant
            Q_load_scenarios[:, idx] = P_load_scenarios[:, idx] * np.tan(np.arccos(power_factor[idx, 0]))

    return P_load_scenarios, Q_load_scenarios

P_load_scenarios_uniform, Q_load_scenarios_uniform = generate_scenarios_uniform(P_load, Q_load, N)

np.save(f"{PATH}/data/P_load_all_scenarios{TEST_CASE}bus_uniform.npy", P_load_scenarios_uniform)
np.save(f"{PATH}/data/Q_load_all_scenarios{TEST_CASE}bus_uniform.npy", Q_load_scenarios_uniform)


In [18]:
import numpy as np
import pandas as pd
import array
import csv
import math
import os.path
import random
import time
from joblib import Parallel, delayed
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
import scipy.io
from pypower.api import ext2int, int2ext, makeYbus
PATH = "/storage/home/hcoda1/6/btaheri6/p-dmolzahn6-0/DistFlow"


TEST_CASE           =   '123'


def load_mat():
    mat = scipy.io.loadmat(f"{PATH}/data/case{TEST_CASE}.mat")
    bus = mat['ans']['bus'][0][0]
    baseMVA = mat['ans']['baseMVA'][0][0][0][0]
    gen = mat['ans']['gen'][0][0]
    branch = mat['ans']['branch'][0][0]
    keys = ['baseMVA', 'bus', 'gen', 'branch']
    values = [baseMVA, bus, gen, branch]
    ppc = dict(zip(keys, values))
    ppc = ext2int(ppc)
    baseMVA, bus, gen, branch = ppc['baseMVA'], ppc['bus'], ppc['gen'], ppc['branch']
    Ybus, Yf, Yt = makeYbus(baseMVA, bus, branch)

    return baseMVA, bus, gen, branch, Ybus, Yf, Yt, ppc


def load_gen_data(filename):
    gen_data = pd.read_csv(filename, sep=',')
    return gen_data


def process_bus_and_gen(bus, gen, baseMVA, branch, ppc):
    Just_load = bus[np.logical_or(bus[:, 2] != 0, bus[:, 3] != 0)]
    load_buses=Just_load[:,0]
    load_buses=np.reshape(load_buses,(len(load_buses),1))

    n_buses = len(ppc['bus'])
    n_gens = len(ppc['gen'])
    n_branches = len(ppc['branch'])
    r = branch[:, 2]
    x = branch[:, 3]
    gen_buses = gen[:, 0]
    gen_values = gen[:, 1]
    Q_values = gen[:, 2]


    Pgen = np.zeros(n_buses)
    Qgen = np.zeros(n_buses)

    for i in range(len(gen_buses)):
        bus_idx = int(gen_buses[i])  
        Pgen[bus_idx] += gen_values[i]
        Qgen[bus_idx] += Q_values[i]

    #Depends on ...    
    Pgen  = Pgen / baseMVA
    Qgen  = Qgen / baseMVA

    Pgen = np.reshape(Pgen,(n_buses,1))    
    Qgen = np.reshape(Qgen,(n_buses,1))    

    ref_bus = np.where(bus[:, 1] == 3)[0][0]

    return load_buses, n_buses, n_gens, n_branches, r, x, Pgen,Qgen, ref_bus


baseMVA, bus, gen, branch, Ybus, Yf, Yt, ppc = load_mat()
load_buses, n_buses, n_gens, n_branches, r, x, Pgen,Qgen, ref_bus = process_bus_and_gen(bus, gen, baseMVA, branch, ppc)

def compute_and_save_pinj():
    
    baseMVA, bus, gen, branch, _, _, _, ppc = load_mat()
    P_load_all_scenarios = np.load(f"{PATH}/data/P_load_all_scenarios_{TEST_CASE}bus_uniform.npy", allow_pickle=True)
    Q_load_all_scenarios = np.load(f"{PATH}/data/Q_load_all_scenarios_{TEST_CASE}bus_uniform.npy", allow_pickle=True)

    load_buses, n_buses, _, _, _, _, Pgen,Qgen, ref_bus = process_bus_and_gen(bus, gen, baseMVA, branch, ppc)

    # Pre-calculate P_inj for all scenarios
    P_inj_all_scenarios = []
    for s in range(P_load_all_scenarios.shape[0]): 
    #for s in range(P_load_all_scenarios.shape[1]-1): 
        aaa = P_load_all_scenarios[s, :]
        #aaa = P_load_all_scenarios.iloc[:, s+1]

        Demand = np.zeros(n_buses)
        for i in range(len(load_buses)):
            #bus_idx = int(load_buses[i])
            bus_idx = int(load_buses[i][0])  # assuming load_buses[i] is a 1D array with one element

            Demand[bus_idx] = aaa[i]
        Demand = np.reshape(Demand, (n_buses, 1))
        P_inj = Pgen - Demand
        P_inj = np.delete(P_inj, ref_bus, 0)
        P_inj = np.asmatrix(P_inj)
        P_inj_all_scenarios.append(P_inj)
        
    Q_inj_all_scenarios = []
    for s in range(Q_load_all_scenarios.shape[0]): 
    #for s in range(Q_load_all_scenarios.shape[1]-1): 
        aaa = Q_load_all_scenarios[s, :]
        #aaa = Q_load_all_scenarios.iloc[:, s+1]

        Demand = np.zeros(n_buses)
        for i in range(len(load_buses)):
            #bus_idx = int(load_buses[i])
            bus_idx = int(load_buses[i][0])  # assuming load_buses[i] is a 1D array with one element

            Demand[bus_idx] = aaa[i]
        Demand = np.reshape(Demand, (n_buses, 1))
        Q_inj = Qgen - Demand
        Q_inj = np.delete(Q_inj, ref_bus, 0)
        Q_inj = np.asmatrix(Q_inj)
        Q_inj_all_scenarios.append(Q_inj)        
        
    # Save to npy file
    np.save(f"{PATH}/data/P_inj_all_scenarios_{TEST_CASE}bus_uniform.npy", P_inj_all_scenarios)
    np.save(f"{PATH}/data/Q_inj_all_scenarios_{TEST_CASE}bus_uniform.npy", Q_inj_all_scenarios)

# Call the function
compute_and_save_pinj()